# Election Simulation
This notebook demonstrates running the election simulation with adjustable poll weights and number of simulations.

In [ ]:
import sys, pathlib, random
root_dir = pathlib.Path().resolve().parent
sys.path.append(str(root_dir))
from election_simulation import State, read_states_info, read_state_list, read_polling_data


In [ ]:
# Parameters
poll_weight_factor = 1.0  # Multiplier applied to each poll weight
simulations = 1000


In [ ]:
# Load data
data_dir = root_dir / "data"

states = read_states_info(data_dir / "states_info.csv")
safe_harris = read_state_list(data_dir / "safe_states_harris.txt")
safe_trump = read_state_list(data_dir / "safe_states_trump.txt")
competitive_states = read_state_list(data_dir / "competitive_states.txt")

for state_name in safe_harris:
    if state_name in states:
        state = states[state_name]
        state.harris_support = 60.0
        state.trump_support = 35.0
        state.std_dev = 2.0
for state_name in safe_trump:
    if state_name in states:
        state = states[state_name]
        state.harris_support = 35.0
        state.trump_support = 60.0
        state.std_dev = 2.0

polls = read_polling_data(data_dir / "polling_data.csv")
for poll in polls:
    state_name = poll['state_name']
    if state_name in states:
        state = states[state_name]
        weight = int(poll['weight'] * poll_weight_factor)
        weight = min(max(weight, 1), 10)
        state.add_poll(
            harris_support=poll['harris_support'],
            trump_support=poll['trump_support'],
            weight=weight,
            moe=poll['moe'],
        )

for state_name in competitive_states:
    if state_name in states:
        states[state_name].calculate_weighted_support()


In [ ]:
# Run simulations
harris_wins = 0
trump_wins = 0
ties = 0

for _ in range(simulations):
    harris_evs = 0
    trump_evs = 0
    for state in states.values():
        margin = state.harris_support - state.trump_support
        result = random.gauss(margin, state.std_dev)
        if result > 0:
            harris_evs += state.electoral_votes
        else:
            trump_evs += state.electoral_votes

    if harris_evs >= 270:
        harris_wins += 1
    elif trump_evs >= 270:
        trump_wins += 1
    else:
        ties += 1

print(f"Out of {simulations} simulations:")
print(f"Kamala Harris wins: {harris_wins} times")
print(f"Donald Trump wins: {trump_wins} times")
print(f"Ties or no majority: {ties} times")
